In [68]:
import pickle
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import random
import json
import os
import re
import seaborn as sns
import math
import sys
import copy

In [69]:
## parameters

caida = ["caida-500w", "caida-250w", "caida-125w", "caida-50w"]
zipf2 = ["zipf2-250w", "zipf2-125w", "zipf2-50w"]
zipf1 = ["zipf1-250w", "zipf1-125w", "zipf1-50w"]


lens = [
        # ["5", "5"],
        ["6", "4"],
        # ["7", "3"],
        # ["8", "2"],
        ]

pcap_file = []
    
# # single dataset
pcap_file.append("caida-500w_10_.pcap")
pcap_file.append("caida-250w_10_.pcap")
pcap_file.append("caida-125w_10_.pcap")
pcap_file.append("caida-50w_10_.pcap")
pcap_file.append("zipf2-250w_10_.pcap") 
pcap_file.append("zipf2-125w_10_.pcap") 
pcap_file.append("zipf2-50w_10_.pcap") 
pcap_file.append("zipf1-250w_10_.pcap") 
pcap_file.append("zipf1-125w_10_.pcap") 
pcap_file.append("zipf1-50w_10_.pcap") 

### caida + zipf2 [10:22]
for a in caida:
    for b in zipf2:
        for l in lens:
            pcap_file.append(f'{a}_{l[0]}_{b}_{l[1]}.pcap')
            
### zipf2 + caida [22:34]
for a in zipf2:
    for b in caida:
        for l in lens:
            pcap_file.append(f'{a}_{l[0]}_{b}_{l[1]}.pcap')
            
### caida + zipf1 [34:46]
for a in caida:
    for b in zipf1:
        for l in lens:
            pcap_file.append(f'{a}_{l[0]}_{b}_{l[1]}.pcap')
            
### zipf1 + caida [46:58]
for a in zipf1:
    for b in caida:
        for l in lens:
            pcap_file.append(f'{a}_{l[0]}_{b}_{l[1]}.pcap')
            
### zipf2 + zipf1 [58:67]
for a in zipf2:
    for b in zipf1:
        for l in lens:
            pcap_file.append(f'{a}_{l[0]}_{b}_{l[1]}.pcap')
            
### zipf1 + zipf2 [67:76]
for a in zipf1:
    for b in zipf2:
        for l in lens:
            pcap_file.append(f'{a}_{l[0]}_{b}_{l[1]}.pcap')
            
### caida + caida [76:88]
for a in caida:
    for b in caida:
        if a == b:
            continue
        for l in lens:
            pcap_file.append(f'{a}_{l[0]}_{b}_{l[1]}.pcap')
            
### zipf2 + zipf2 [88:94]
for a in zipf2:
    for b in zipf2:
        if a == b:
            continue
        for l in lens:
            pcap_file.append(f'{a}_{l[0]}_{b}_{l[1]}.pcap')
            
### zipf1 + zipf1 [94:100]
for a in zipf1:
    for b in zipf1:
        if a == b:
            continue
        for l in lens:
            pcap_file.append(f'{a}_{l[0]}_{b}_{l[1]}.pcap')
            
print(pcap_file)    
print(f'Total Pcap File Number: {len(pcap_file)}')
# widths = [2048, 4096, 8192, 16384, 32768, 65536, 131072]
widths = [4096]
# widths = [1024]



['caida-500w_10_.pcap', 'caida-250w_10_.pcap', 'caida-125w_10_.pcap', 'caida-50w_10_.pcap', 'zipf2-250w_10_.pcap', 'zipf2-125w_10_.pcap', 'zipf2-50w_10_.pcap', 'zipf1-250w_10_.pcap', 'zipf1-125w_10_.pcap', 'zipf1-50w_10_.pcap', 'caida-500w_6_zipf2-250w_4.pcap', 'caida-500w_6_zipf2-125w_4.pcap', 'caida-500w_6_zipf2-50w_4.pcap', 'caida-250w_6_zipf2-250w_4.pcap', 'caida-250w_6_zipf2-125w_4.pcap', 'caida-250w_6_zipf2-50w_4.pcap', 'caida-125w_6_zipf2-250w_4.pcap', 'caida-125w_6_zipf2-125w_4.pcap', 'caida-125w_6_zipf2-50w_4.pcap', 'caida-50w_6_zipf2-250w_4.pcap', 'caida-50w_6_zipf2-125w_4.pcap', 'caida-50w_6_zipf2-50w_4.pcap', 'zipf2-250w_6_caida-500w_4.pcap', 'zipf2-250w_6_caida-250w_4.pcap', 'zipf2-250w_6_caida-125w_4.pcap', 'zipf2-250w_6_caida-50w_4.pcap', 'zipf2-125w_6_caida-500w_4.pcap', 'zipf2-125w_6_caida-250w_4.pcap', 'zipf2-125w_6_caida-125w_4.pcap', 'zipf2-125w_6_caida-50w_4.pcap', 'zipf2-50w_6_caida-500w_4.pcap', 'zipf2-50w_6_caida-250w_4.pcap', 'zipf2-50w_6_caida-125w_4.pcap', 'z

### TFS

In [70]:
def read_total_size_data(algo='cm', row=3, width=4096, level=1, seed=1, count=1, flowkey='srcIP', 
              epochs=['10'], dataset='caida20180517_10_caida20180816_0.pcap', window_size=200):
    res = []
    for epoch in epochs:
        
        path = f"../SketchPatternQuery/{algo}/{dataset}/"\
                f"{flowkey}/row_{row}_width_{width}_level_{level}_epoch_{epoch}_count_{count}_seed_{seed}/"
        
        for dir in sorted(os.listdir(path)):
            p = os.path.join(path, dir)
            if os.path.isdir(p): 
                window_dir = "window_" + str(window_size)
                full_path = os.path.join(path, dir, window_dir,'total_flow_size.txt')    
                with open(full_path, 'r') as f:
                    for val in f:
                        res.append(int(val))
            
    return res

# read_total_size_data()

In [71]:
def tfs_changes_occured(var, sec_var, n, ws=20):
    
    time_var = -1.0
    time_sec_var = -1.0
    
    val = 0
    max_time = -1.0
    
    # for each sliding window
    for i in range(len(var) - ws + 1):
        var_mean = np.mean(var[0+i:ws+i])
        var_std_dev = np.std(var[0+i:ws+i])
        
        # check by var outliers
        if abs(var[ws+i-1] - var_mean) > (n * var_std_dev):
            if time_var == -1.0:
                time_var = (1+(ws+i-1)-1)/5
                break
                
    for i in range(len(sec_var) - ws + 1):            
        sec_var_mean = np.mean(sec_var[0+i:ws+i])
        sec_var_std_dev = np.std(sec_var[0+i:ws+i])
        
        # check by sec var outliers
        if abs(sec_var[ws+i-1] - sec_var_mean) > (n * sec_var_std_dev):
            if time_sec_var == -1.0:
                time_sec_var = (2+(ws+i-1)-1)/5
                val = sec_var[ws+i-1]
                break
                
    if val == 0:
        val = max(sec_var)
        max_time = (2 + sec_var.index(val) - 1)/5
        
    
    var_hit = 0
    sec_var_hit = 0
    
    if time_var != -1.0:
        var_hit = 1
    if time_sec_var != -1.0:
        sec_var_hit = 1
    
    return var_hit, time_var, sec_var_hit, time_sec_var, val, max_time
    
    

In [72]:
def pcd_tfs(algo='cm', row=3, width=[4096], level=1, seed=1, count=1, flowkey='srcIP', 
              epochs=['10'], datasets=['caida-500w_10_.pcap'], window_size=200, dev=3, ws=25):
    
    # total flow size
    res_sum = {}
    res_var = {}
    res_sec_var = {}
    
    label = []
    answer = []
    
    for d in datasets:
        for w in width:
            name = f'{d[:-5]} {w}'
            res_var[name] = [None]
            res_sec_var[name] = [None, None]
            
            res_total = read_total_size_data(algo, row, w, level, seed, count, flowkey, epochs, d, window_size)

            label.append(name)
            answer.append(int(d.split('_')[1]))
            # answer.append(10)
            
            res_sum[name] = res_total
            
            for i in range(1, len(res_total)):
                res_var[name].append(res_total[i] - res_total[i-1])
                
            for i in range(2, len(res_var[name])):
                res_sec_var[name].append(abs(res_var[name][i] - res_var[name][i-1]))
    
    PCD_TFS = {}
    for d in datasets:
        PCD_TFS[d[:-5]] = {}
        for w in width:
            PCD_TFS[d[:-5]][w] = (0, 0)
    
    for i in range(len(label)):
        
        var_hit, time_var, sec_var_hit, time_sec_var, val, max_time = tfs_changes_occured(res_var[label[i]][1:], res_sec_var[label[i]][2:], dev, ws)
        
        d = label[i].split(" ")[0]
        w = int(label[i].split(" ")[1])
        
        if var_hit == 0:
            PCD_TFS[d][w] = (0, 0)
        else:
            PCD_TFS[d][w] = (1, int(time_sec_var/(window_size/1000)))
            
    return PCD_TFS

In [101]:
def epi_tfs(algo='cm', row=3, width=[4096], level=1, seed=1, count=1, flowkey='srcIP', epochs=['10'], 
                datasets=['caida-500w_10_.pcap'], window_size=200, PCD_TFS={}, predict_time=[0.2, 1.0, 2.0]):
    
    answer = {5000000: '500w',
              2500000: '250w', 
              1250000: '125w', 
              500000: '50w',
              }
    
    
    EPI_TFS = {}
    for d in datasets:
        EPI_TFS[d[:-5]] = {}
        for w in width:
            EPI_TFS[d[:-5]][w] = {}
            for t in predict_time:
                EPI_TFS[d[:-5]][w][t] = ""


    for d in datasets:
        for w in width:
            for t in predict_time:
                if PCD_TFS[d[:-5]][w][0] == 0: # no pattern change
                    EPI_TFS[d[:-5]][w][t] = d.split("_")[0].split("-")[1]
                else:
                    start_time = PCD_TFS[d[:-5]][w][1]
                    result = read_total_size_data(algo, row, w, level, seed, count, flowkey, epochs, d, window_size)[start_time:]
    
                    # variation
                    res = []
                    fin_res_var = [None]
                    for i in range(1, len(result)):
                        fin_res_var.append(result[i] - result[i-1])    
                    res.append(fin_res_var)
                    
                    predict_tfs = sum(fin_res_var[1:min(len(fin_res_var), 1+int(t*1000/window_size))]) * (10/min((len(fin_res_var)-1)/5, t))
                    closest_key = min(answer.keys(), key=lambda x: abs(x - predict_tfs))
                    EPI_TFS[d[:-5]][w][t] = answer[closest_key]
    
    return EPI_TFS
    

### FSD

In [74]:
def read_single_window_fsd_data(algo='cm', row=3, width=4096, level=1, seed=1, count=1, flowkey='srcIP', 
              epochs=['10'], dataset='caida0517-125w_10_.pcap', window_size=200, k=5000):
    res = []
    for epoch in epochs:
        
        path = f"../SketchPatternQuery/{algo}/{dataset}/"\
                f"{flowkey}/row_{row}_width_{width}_level_{level}_epoch_{epoch}_count_{count}_seed_{seed}/"
        
        for dir in sorted(os.listdir(path)):
            p = os.path.join(path, dir)
            if os.path.isdir(p): 
                window_dir = "window_" + str(window_size)
                dynamic_full_path = os.path.join(path, dir, window_dir, f"top{k}/single_window_randk_summation")
                
                for file in sorted(os.listdir(dynamic_full_path)):  
                    fsd_file = os.path.join(dynamic_full_path, file)
                    fsd = {}
                    with open(fsd_file, 'r') as f:
                        for line in f:
                            if int(line.strip().split()[0]) == 0:
                                continue
                            fsd[int(line.strip().split()[0])] = int(line.strip().split()[1])
                            
                    res.append(fsd)
            
    return res

# read_fsd_data()

In [75]:
def fsd_changes_occured(sec_var, n, ws=25):
    
    time_sec_var = -1.0
    
    val = 0
    max_time = -1.0
    
    # for each sliding window           
    for i in range(len(sec_var) - ws + 1):            
        sec_var_mean = np.mean(sec_var[0+i:ws+i])
        sec_var_std_dev = np.std(sec_var[0+i:ws+i])
        
        # check by sec var outliers
        if abs(sec_var[ws+i-1] - sec_var_mean) > (n * sec_var_std_dev):
            if time_sec_var == -1.0:
                time_sec_var = (2+(ws+i-1)-1)/5
                val = sec_var[ws+i-1]
                break
                
    if val == 0:
        val = max(sec_var)
        max_time = (4 + sec_var.index(val) - 1)/5
        
    sec_var_hit = 0
    
    if time_sec_var != -1.0:
        sec_var_hit = 1
    
    return sec_var_hit, time_sec_var, val, max_time
    
    

In [76]:
def calculate_mrd(fsd1, fsd2):
    MRD_nom = 0
    MRD_denom = 0
    for i in range(1, max(fsd1.keys())+1):
        if i in fsd1.keys():
            true = fsd1[i]
        else:
            true = 0
            
        if i in fsd2.keys():
            est = fsd2[i]
        else:
            est = 0
            
        MRD_nom += abs(true - est)
        MRD_denom += float(true + est)/2
    MRD = MRD_nom/MRD_denom
    
    return MRD

In [77]:
def normalize_fsd(fsd):
    fn = sum(list(fsd.values()))
    
    return {key: (val/fn) for key,val in fsd.items()}

In [78]:
def prepare_profiler_dirty_fsd_data(algo='cm', row=3, width=4096, level=1, seed=1, count=1, flowkey='srcIP', 
              epochs=['10'], dataset='caida0517-125w_10_.pcap', window_size=200, k=5000):
    
    profiler_fsd = {}
    profiler_folder_path = "/home/ming/SketchMercator/pattern_detection/traffic_generator/pcap_file_new/"
    for file_name in sorted(os.listdir(profiler_folder_path)):
        profiler_fsd[file_name[:-9]] = []
    
    # res = {}
    for epoch in epochs:
        for pf in profiler_fsd.keys():
            path = f"../SketchPatternQueryOffline/{algo}/{pf}_10_.pcap/"\
                    f"{flowkey}/row_{row}_width_{width}_level_{level}_epoch_{epoch}_count_{count}_seed_{seed}/"
            
            for dir in sorted(os.listdir(path)):
                p = os.path.join(path, dir)
                if os.path.isdir(p): 
                    window_dir = "window_" + str(window_size)
                    dynamic_full_path = os.path.join(path, dir, window_dir, f"top{k}/single_window_randk_summation")
                    
                    for file in sorted(os.listdir(dynamic_full_path)):  
                        fsd_file = os.path.join(dynamic_full_path, file)
                        fsd = {}
                        with open(fsd_file, 'r') as f:
                            for line in f:
                                if int(line.strip().split()[0]) == 0:
                                    continue
                                fsd[int(line.strip().split()[0])] = int(line.strip().split()[1])
                                
                        profiler_fsd[pf].append(fsd)
                
    return profiler_fsd

# read_fsd_data()

In [79]:
def get_avg_fsd(fsd_list):
    avg_fsd = {}
    
    all_keys = set().union(*[d.keys() for d in fsd_list])
    for key in all_keys:
        avg_fsd[key] = 0
        
    for fsd in fsd_list:
        for key, val in fsd.items():
            avg_fsd[key] += val
            
    avg_fsd = {key: int(val/len(fsd_list)) for key, val in avg_fsd.items()}
    sorted_fsd = dict(sorted(list(avg_fsd.items())))
    
    return sorted_fsd


In [80]:
def find_dist_by_dirty_fsd(profiler_fsd, unknown_fsd):
    
    all_mrd = {}
    min_mrd = sys.float_info.max
    predict_dist = ""
    for name, dist in profiler_fsd.items():
        # mrd = calculate_mrd(dist, unknown_fsd)
        mrd = calculate_mrd(normalize_fsd(dist), normalize_fsd(unknown_fsd))
        all_mrd[name] = mrd
        if mrd < min_mrd:
            min_mrd = mrd
            predict_dist = name
    
    mrds = []
    for name, mrd in all_mrd.items():
        mrds.append(mrd)
        
    return predict_dist, mrds
    

In [81]:
def pcd_dist(algo='cm', row=3, width=[4096], level=1, seed=1, count=1, flowkey='srcIP', 
              epochs=['10'], datasets=['caida-125w_10_.pcap'], window_size=200, dev=3, ws=25,k=5000):
    
    res_fsd = {}
    res_mrd = {}
    
    label = []
    answer = []
    
    for d in datasets:
        for w in width:
            name = f'{d[:-5]} {w}'
            res_mrd[name] = [None, None] # 0, 1
            
            fsd_total = read_single_window_fsd_data(algo, row, w, level, seed, count, flowkey, epochs, d, window_size,k)

            label.append(name)
            answer.append(int(d.split('_')[1]))
            # answer.append(10)
            
            res_fsd[name] = fsd_total[:-1] # ignore last window (less than 200ms)
            
            for i in range(1, len(res_fsd[name])):
                # res_mrd[name].append(calculate_mrd(res_fsd[name][i-1], res_fsd[name][i]))
                res_mrd[name].append(calculate_mrd(normalize_fsd(res_fsd[name][i-1]), normalize_fsd(res_fsd[name][i])))
                
    
    PCD_DIST = {}
    for d in datasets:
        PCD_DIST[d[:-5]] = {}
        for w in width:
            PCD_DIST[d[:-5]][w] = (0, 0)

    
    for i in range(len(label)):
        
        var_hit, time_var, val, max_time = fsd_changes_occured(res_mrd[label[i]][2:], dev, ws)
        
        d = label[i].split(" ")[0]
        w = int(label[i].split(" ")[1])
        
        if var_hit == 0:
            PCD_DIST[d][w] = (0, 0)
        else:
            PCD_DIST[d][w] = (1, int(time_var/(window_size/1000)))
            
    return PCD_DIST
    

In [82]:
def epi_dist(algo='cm', row=3, width=[4096], level=1, seed=1, count=1, flowkey='srcIP', 
              epochs=['10'], datasets=['caida-125w_10_.pcap'], window_size=200, PCD_DIST={}, predict_time=[0.2, 1.0, 2.0], k=5000):
    
    res_fsd = {}
    
    label = []
    answer = []
    
    for d in datasets:
        for w in width:
            name = f'{d[:-5]} {w}'
            
            fsd_total = read_single_window_fsd_data(algo, row, w, level, seed, count, flowkey, epochs, d, window_size, k)

            label.append(name)
            answer.append(int(d.split('_')[1]))
            # answer.append(10)
            
            res_fsd[name] = fsd_total
            
    
    # find dist
    success = 0
    profiler_fsd = prepare_profiler_dirty_fsd_data(k=k)
    # prepare profiler avg dirty fsd
    profiler_avg_fsd = {}
    for name, fsd_list in profiler_fsd.items():
        profiler_avg_fsd[name] = get_avg_fsd(fsd_list)
        
    EPI_DIST = {}
    for d in datasets:
        EPI_DIST[d[:-5]] = {}
        for w in width:
            EPI_DIST[d[:-5]][w] = {}
            for t in predict_time:
                EPI_DIST[d[:-5]][w][t] = ""
        
    for i in range(len(label)):
        d = label[i].split(" ")[0]
        w = int(label[i].split(" ")[1])
        
        for t in predict_time:
            if PCD_DIST[d][w][0] == 0: # no pattern change
                EPI_DIST[d][w][t] = d.split("_")[0].split("-")[0]
            else:
                start_time = PCD_DIST[d][w][1]
                
                vote = {}
            
                ### vote
                for j in range(min(int(t * 1000 / window_size), len(res_fsd[label[i]])-start_time)):
                    # print(start_time + j)
                    dist_name, mrds = find_dist_by_dirty_fsd(profiler_avg_fsd, res_fsd[label[i]][start_time+j])
                    # print(dist_name)
                    if dist_name in vote:
                        vote[dist_name] += 1
                    else:
                        vote[dist_name] = 1
                        
                EPI_DIST[d][w][t] = max(vote, key=vote.get).split("-")[0]
            
    return EPI_DIST
 

### Final Predict

In [89]:
def config_mapping(dataset):
    config = {}
    config["caida-500w"] = "cs"
    config["caida-250w"] = "cs"
    config["caida-125w"] = "cm"
    config["caida-50w"] = "cm"
    config["zipf2-250w"] = "cs"
    config["zipf2-125w"] = "cm"
    config["zipf2-50w"] = "cm"
    config["zipf1-250w"] = "cm"
    config["zipf1-125w"] = "cm"
    config["zipf1-50w"] = "cm"
    
    if dataset not in config.keys():
        return "Error"
    
    return config[dataset]

In [103]:
def pattern_sensing(algo='cm', row=3, width=[4096], level=1, seed=1, count=1, flowkey='srcIP', 
              epochs=['10'], datasets=['caida0517-125w_10_.pcap'], window_size=200, dev=3, ws=20, k=5000 , predict_time=[0.2, 1.0, 2.0]):
    
    # prepare time series data
    # TFS
    PCD_TFS = pcd_tfs(algo, row, width, level, seed, count, flowkey, epochs, datasets, window_size, dev, ws)
    EPI_TFS = epi_tfs(algo, row, width, level, seed, count, flowkey, epochs, datasets, window_size, PCD_TFS, predict_time)
    
    # DIST
    PCD_DIST = pcd_dist(algo, row, width, level, seed, count, flowkey, epochs, datasets, window_size, dev, ws)
    EPI_DIST = epi_dist(algo, row, width, level, seed, count, flowkey, epochs, datasets, window_size, PCD_DIST, predict_time, k)
    
    acc = [0] * len(predict_time)
    cfg_changed_acc = [0] * len(predict_time)
    cfg_nochanged_acc = [0] * len(predict_time)
    cfg_changed_num = [0] * len(predict_time)
    cfg_nochanged_num = [0] * len(predict_time)
    
    for d in datasets:
        for w in width:
            print(d)
            print(f'\tTFS changing time:  {PCD_TFS[d[:-5]][w][1]*0.2}s')
            print(f'\tDIST changing time: {PCD_DIST[d[:-5]][w][1]*0.2}s')
            
            previous_pattern = d.split("_")[0]
            current_pattern = (d.split("_")[0] if "10" in d else d.split("_")[2])
            print(f'\t      Current Pattern: {current_pattern}')
            for i, t in enumerate(predict_time):
                predict_pattern = f'{EPI_DIST[d[:-5]][w][t]}-{EPI_TFS[d[:-5]][w][t]}'
                print(f'\tt={t} Predict Pattern: {predict_pattern}')
                
                # cal acc
                if current_pattern == predict_pattern:
                    acc[i] += 1

                if config_mapping(previous_pattern) == config_mapping(current_pattern): # config no changed
                    cfg_nochanged_num[i] += 1
                    if config_mapping(current_pattern) == config_mapping(predict_pattern):
                        cfg_nochanged_acc[i] += 1
                    else:
                        print("[failed] no change")
                else: # config changed
                    cfg_changed_num[i] += 1
                    if config_mapping(current_pattern) == config_mapping(predict_pattern):
                        cfg_changed_acc[i] += 1
                    else:
                        print("[failed] change")
            print()
                
    print()
    for i, t in enumerate(predict_time):
        print(f't={t}')
        print(f'\t acc      ={acc[i]}/100')
        print(f'\t no change={cfg_nochanged_acc[i]}/{cfg_nochanged_num[i]}')
        print(f'\t change   ={cfg_changed_acc[i]}/{cfg_changed_num[i]}')
        
    return

pattern_sensing(datasets=pcap_file, predict_time=[0.2])
pattern_sensing(datasets=pcap_file, predict_time=[1])
pattern_sensing(datasets=pcap_file, predict_time=[2])

caida-500w_10_.pcap
	TFS changing time:  7.4s
	DIST changing time: 0.0s
	      Current Pattern: caida-500w
	t=0.2 Predict Pattern: caida-500w

caida-250w_10_.pcap
	TFS changing time:  9.4s
	DIST changing time: 0.0s
	      Current Pattern: caida-250w
	t=0.2 Predict Pattern: caida-125w
[failed] no change

caida-125w_10_.pcap
	TFS changing time:  -1.0s
	DIST changing time: 0.0s
	      Current Pattern: caida-125w
	t=0.2 Predict Pattern: caida-125w

caida-50w_10_.pcap
	TFS changing time:  0.0s
	DIST changing time: 0.0s
	      Current Pattern: caida-50w
	t=0.2 Predict Pattern: caida-50w

zipf2-250w_10_.pcap
	TFS changing time:  0.0s
	DIST changing time: 0.0s
	      Current Pattern: zipf2-250w
	t=0.2 Predict Pattern: zipf2-250w

zipf2-125w_10_.pcap
	TFS changing time:  0.0s
	DIST changing time: 7.800000000000001s
	      Current Pattern: zipf2-125w
	t=0.2 Predict Pattern: zipf2-125w

zipf2-50w_10_.pcap
	TFS changing time:  7.2s
	DIST changing time: 0.0s
	      Current Pattern: zipf2-50w
	t=0.2